In [ ]:
import pandas as pd
import os
import glob
from tqdm.auto import tqdm
import re

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install libpoppler-cpp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 50.4 kB of archives.
After this operation, 231 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp0v5 amd64 22.02.0-2ubuntu0.2 [38.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp-dev amd64 22.02.0-2ubuntu0.2 [11.8 kB]
Fetched 50.4 kB in 1s (94.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debc

In [ ]:
!sudo apt-get install libpoppler58=0.41.0-0ubuntu1 libpoppler-dev libpoppler-cpp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libpoppler58


In [ ]:
pip install pdftotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pdftotext: filename=pdftotext-2.2.2-cp310-cp310-linux_x86_64.whl size=61849 sha256=b68a2002427f734c6d8fec50616c3ddd978abdb2df30675cf6330e35d08526cd
  Stored in directory: /root/.cache/pip/wheels/c8/68/c6/82a8c2046e1a8f9051017319dc007f43ef3a6edc53149d1f4e
Successfully built pdftotext


In [ ]:
#to clean the accents
from unidecode import unidecode

def cleanThePage(page):
  page = unidecode(page)

  page = page.lower()

  #Removing the first and line of a page
  page = '\n'.join(page.split('\n')[1:-1])

  #removing In [34]: and Out [45]:  (jupyter notebook input and output)
  page = re.sub('In\s.\d*..', '', page)

  #removing hyphenated, and underscore lines lines
  page = re.sub('[-]{5,}', '', page)
  page = re.sub('[_]{5,}', '', page)

  #Removing a hyperlink
  page = re.sub(r'https?:\/\/.*[\r\n]*', '', page)

  #Handling all date formats
  page = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', page)
  page = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', page)
  page = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', page)

  #Removing numerical data
  #re.sub(r'\d+','','I have 500 rupees with me.')

  # Replacing 2 or more newlines with a single one
  page = re.sub(r'\n{2,}','\n', page)

  #Replacing 2 or more whitespaces with a single one to make word spacing uniform
  page = re.sub(r' {2,}',' ', page)

  #removing the error strings enclosed in angular brackets
  page = re.sub(r'<.*?>', '', page)

  page = re.sub(r'Traceback .most recent call last.', '', page)

  #removal of prompt
  page = re.sub(r'>>>', '', page)

  page = re.sub(r'[\.\s]+', ' ',page)  #to remove . . .
  page = re.sub(r'[Cc]hapter.\d*',"",page) #to remove chapter numbers
  page = re.sub(r'\xa0', '', page)
  page = re.sub(r'\x06', '', page)
  page = re.sub(r'\x02', '', page)

  page = re.sub(r'[^\w\s] ', '', page)

  #removing digits
  page = re.sub(r'\d', " ", page)

  page = re.sub(r'\b\w\b',' ', page) #match single characters separated by spaces

  page = re.sub(r' {2,}',' ', page)

  page = page.strip()

  return page

In [ ]:
cleanThePage('\n\na b c d e f      89899sairam is my favourite\nHello           python       \n\n')

'sairam is my favourite hello python'

In [ ]:
import re
str = 'a b dfkAdfkd ent f    dfhkfd\nsdfjd'
str = str.lower()
re.sub(r'\s[A-Za-z0-9]\s',' ', str)

'a dfkadfkd ent    dfhkfd\nsdfjd'

In [ ]:
textbooks_path = '/content/drive/MyDrive/Personal/Unique Books'
file_paths = glob.glob(os.path.join(textbooks_path,  '*'))

In [ ]:
len(file_paths)

99

In [ ]:
file_paths[10]

'/content/drive/MyDrive/Personal/Unique Books/15phillipos_PythonObjectOrientedProgramming_2ed.pdf'

In [ ]:
import pdftotext
##initialization of the id
idx = 0
#dataset of dictionaries
data = []

for file in tqdm(file_paths):
  # Load your PDF associated with the file path
  with open(file, "rb") as f:
    pdf = pdftotext.PDF(f)
    #print(file, "--", len(pdf))
    #Stripping the first and last few pages based on the number of pages in the book
    pagecount = len(pdf)
    if pagecount > 100:
      start, stop  = 5, pagecount - 5
    elif pagecount >10:
      start, stop = 2, pagecount -2
    else:
      start, stop = 0, pagecount

    #extract every page, clean it, and put it in dictionary
    for i in range(start, stop):
      #create a dictionary
      if len(pdf[i]) > 20:  #if the page contains atleast 20 characters
        data.append({'id' : idx, 'text': cleanThePage(pdf[i])})
        idx = idx+1

  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
len(data)  #number of pages

25572

In [ ]:
data[10552]

{'id': 10552,
 'text': "drawing images with pygame image load(and blit(the drawing functions are fine if you want to draw simple shapes on the screenbut many games have images (also called spritespygame is able to load images onto surface objects from pngjpggifand bmp image files the differences between these image file formats is described at the image of the cat was stored in file named cat png to load this file' imagethe string 'cat pngis passed to the pygame image load(function the pygame image load(function call will return surface object that has the image drawn on it this surface object will be separate surface object from the display surface objectso we must blit (that iscopythe image' surface object to the display surface object blitting is drawing the contents of one surface onto another it is done with the blit(surface object method if you get an error message like --pygame errorcouldn' open cat png|when calling pygame image load()then make sure the cat png file is in the sa

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 9.5 MB/s eta 0:00:00


In [ ]:
import datasets
from datasets import load_dataset
tbdata = datasets.Dataset.from_pandas(pd.DataFrame(data=data))

In [ ]:
tbdata

Dataset({
    features: ['id', 'text'],
    num_rows: 25572
})

In [ ]:
tbdata.save_to_disk("tbdata")

Saving the dataset (0/1 shards):   0%|          | 0/25572 [00:00<?, ? examples/s]

In [ ]:
dataset = datasets.load_from_disk("tbdata")

In [ ]:
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 25572
})

In [ ]:
dataset[120]

{'id': 120,
 'text': 'to get the value associated with keygive the name of the dictionary and then place the key inside set of square bracketsas shown herealien_ {\'color\'\'green\'print(alien_ [\'color\']this returns the value associated with the key \'colorfrom the dictionary alien_ green you can have an unlimited number of key-value pairs in dictionary for examplehere\' the original alien_ dictionary with two key-value pairsalien_ {\'color\'\'green\'\'points\' now you can access either the color or the point value of alien_ if player shoots down this alienyou can look up how many points they should earn using code like thisalien_ {\'color\'\'green\'\'points\' new_points alien_ [\'points\' print("you just earned str(new_pointspoints!"once the dictionary has been definedthe code at pulls the value associated with the key \'pointsfrom the dictionary this value is then stored in the variable new_points the line at converts this integer value to string and prints statement about how many

In [ ]:
from huggingface_hub import login

In [ ]:
login()

In [ ]:
dataset.push_to_hub("Sridevi/python_textbooks")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset[9000]

{'id': 9000,
 'text': 'inside the try block we get the lambda function that is appropriate to the requested action we use lock to protect access to the call dictionaryalthough arguably we are being overly cautious as alwayswe do as little as possible within the scope of the lock--in this case we just do dictionary lookup to get reference to function once we have the function we call itpassing self as the first argument and the rest of the data tuple as the other arguments here we are doing function callso no self is passed by python this does not matter since we pass self in ourselvesand inside the lambda the passed-in self is used to call the method in the normal way the outcome is that the callself method(*data[ :])is madewhere method is the method corresponding to the action given in data[ if the action is to shut downa custom finish exception is raised in the shutdown(methodin which case we know that the client cannot expect replyso we just return but for any other action we pickle

In [ ]:
str = dataset[3899]['text']

In [ ]:
str

'solving ordinary differential equations cfor the case in )find through experimentation the largest value of where the exact solution and the numerical solution by heun\' method cannot be distinguished visually it is of interest to see how far off the curve the forward euler method is when heun\' method can be regarded as "exact(for visual purposesfilenameode_heun py exercise find an appropriate time steplogistic model compute the numerical solution of the logistic equation for set of repeatedly halved time stepstk tk plot the solutions corresponding to the last two time steps tk and tk in the same plot continue doing this until you cannot visually distinguish the two curves in the plot then one has found sufficiently small time step hint extend the logistic py file introduce loop over kwrite out tk and ask the user if the loop is to be continued filenamelogistic_dt py exercise find an appropriate time stepsir model repeat exercise for the sir model hint import the ode_fe function from